In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import matplotlib.pyplot as plt
from collections import deque
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
import random
import keras
from keras.layers import *
from keras.models import *

Creating Random Plays

In [2]:
# Create Env
# env_2 = gym.make('MountainCar-v0')
# for i_episode in range(20):
#     observation = env_2.reset()
#     for t in range(100):
#         env_2.render()
#         #print(observation)
#         action = env_2.action_space.sample()
#         observation, reward, done, info = env_2.step(action)
        
#         if done:
#             print(f"Episode:{i_episode + 1}/20, Reward:{reward}")
#             break
# env_2.close()

In [3]:
class Agent:
    def __init__(self,state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95 #Discount Factor
        
        #exploration vs exploitation tradeoff
        self.epsilon = 1.0 #100% exploration at the beginning
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self._create_model()
        
    def _create_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim = self.state_size , activation = 'relu'))
        model.add(Dense(24 , activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'linear'))
        model.compile(loss = 'mse' , optimizer = Adam( lr = 0.001))
        #model.summary()
        return model
    
    def remember(self , state, action, reward, next_state, done):
        #put the above parameters in memory for training..
        self.memory.append((state, action, reward, next_state, done))
        
    def act( self, state):
        #epsilon greedy method...
        if np.random.rand() <= self.epsilon:
            #Take random action 
            return random.randrange(self.action_size)
        
        #Ask NN to give suitable action
        return np.argmax(self.model.predict(state)[0])
    
    def train(self, batch_size = 32):
        
        minibatch = random.sample(self.memory, batch_size)
        for exp in minibatch:
            state, action, reward, next_state, done = exp
            
            if not done:
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
                
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state, target_f, epochs = 1 , verbose = 0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self, name):
        self.model.load_weights(name)
        
    def save(self , name):
        self.model.save_weights(name)
                
                
    

In [4]:
n_episodes = 1000
env = gym.make('CartPole-v0')

In [5]:
agent = Agent(state_size = 4, action_size = 2)
done = False
state_size = 4
action_size = 2

C:\Users\Troy_Hawk\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
for e in range( n_episodes):
    state = env.reset()
    state = np.reshape(state , [1, state_size])
    batch_size = 32
    
    for time in range(500):
        env.render()
        action = agent.act(state)
        
        next_state, reward, done, other_info = env.step(action)
#         print(state)
#         print(state.shape)
#         print(next_state)
#         print(next_state.shape)
        reward = -100*(abs(next_state[2]) - abs(state[0,2])) 
        next_state = np.reshape( next_state , [1, state_size])
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        
        if done:
            print(f"Episode {e + 1}/500 finished after {time+1} timesteps. Exploration Rate = {agent.epsilon}")
            break
    if len(agent.memory) > batch_size:
        agent.train(batch_size)
    
print("Deep Q-Learner Model Trained")
env.close()
            

Episode 1/500 finished after 28 timesteps. Exploration Rate = 1.0
Episode 2/500 finished after 29 timesteps. Exploration Rate = 1.0
Episode 3/500 finished after 9 timesteps. Exploration Rate = 0.995
Episode 4/500 finished after 25 timesteps. Exploration Rate = 0.990025
Episode 5/500 finished after 12 timesteps. Exploration Rate = 0.985074875
Episode 6/500 finished after 36 timesteps. Exploration Rate = 0.9801495006250001
Episode 7/500 finished after 9 timesteps. Exploration Rate = 0.9752487531218751
Episode 8/500 finished after 16 timesteps. Exploration Rate = 0.9703725093562657
Episode 9/500 finished after 22 timesteps. Exploration Rate = 0.9655206468094844
Episode 10/500 finished after 12 timesteps. Exploration Rate = 0.960693043575437
Episode 11/500 finished after 9 timesteps. Exploration Rate = 0.9558895783575597
Episode 12/500 finished after 18 timesteps. Exploration Rate = 0.9511101304657719
Episode 13/500 finished after 13 timesteps. Exploration Rate = 0.946354579813443
Episode 

Episode 102/500 finished after 85 timesteps. Exploration Rate = 0.6057704364907278
Episode 103/500 finished after 134 timesteps. Exploration Rate = 0.6027415843082742
Episode 104/500 finished after 62 timesteps. Exploration Rate = 0.5997278763867329
Episode 105/500 finished after 17 timesteps. Exploration Rate = 0.5967292370047992
Episode 106/500 finished after 63 timesteps. Exploration Rate = 0.5937455908197752
Episode 107/500 finished after 24 timesteps. Exploration Rate = 0.5907768628656763
Episode 108/500 finished after 28 timesteps. Exploration Rate = 0.5878229785513479
Episode 109/500 finished after 34 timesteps. Exploration Rate = 0.5848838636585911
Episode 110/500 finished after 68 timesteps. Exploration Rate = 0.5819594443402982
Episode 111/500 finished after 116 timesteps. Exploration Rate = 0.5790496471185967
Episode 112/500 finished after 27 timesteps. Exploration Rate = 0.5761543988830038
Episode 113/500 finished after 39 timesteps. Exploration Rate = 0.5732736268885887
Ep

Episode 201/500 finished after 89 timesteps. Exploration Rate = 0.36880183088056995
Episode 202/500 finished after 146 timesteps. Exploration Rate = 0.3669578217261671
Episode 203/500 finished after 161 timesteps. Exploration Rate = 0.36512303261753626
Episode 204/500 finished after 200 timesteps. Exploration Rate = 0.3632974174544486
Episode 205/500 finished after 200 timesteps. Exploration Rate = 0.3614809303671764
Episode 206/500 finished after 200 timesteps. Exploration Rate = 0.3596735257153405
Episode 207/500 finished after 94 timesteps. Exploration Rate = 0.3578751580867638
Episode 208/500 finished after 200 timesteps. Exploration Rate = 0.35608578229633
Episode 209/500 finished after 200 timesteps. Exploration Rate = 0.3543053533848483
Episode 210/500 finished after 200 timesteps. Exploration Rate = 0.35253382661792404
Episode 211/500 finished after 132 timesteps. Exploration Rate = 0.3507711574848344
Episode 212/500 finished after 120 timesteps. Exploration Rate = 0.3490173016

Episode 299/500 finished after 200 timesteps. Exploration Rate = 0.22566020663225933
Episode 300/500 finished after 200 timesteps. Exploration Rate = 0.22453190559909803
Episode 301/500 finished after 200 timesteps. Exploration Rate = 0.22340924607110255
Episode 302/500 finished after 200 timesteps. Exploration Rate = 0.22229219984074702
Episode 303/500 finished after 200 timesteps. Exploration Rate = 0.2211807388415433
Episode 304/500 finished after 200 timesteps. Exploration Rate = 0.22007483514733558
Episode 305/500 finished after 200 timesteps. Exploration Rate = 0.2189744609715989
Episode 306/500 finished after 200 timesteps. Exploration Rate = 0.2178795886667409
Episode 307/500 finished after 200 timesteps. Exploration Rate = 0.2167901907234072
Episode 308/500 finished after 200 timesteps. Exploration Rate = 0.21570623976979014
Episode 309/500 finished after 200 timesteps. Exploration Rate = 0.21462770857094118
Episode 310/500 finished after 200 timesteps. Exploration Rate = 0.21

KeyboardInterrupt: 